In [1]:
from PIL import Image

In [2]:
from torchvision import transforms, datasets
from glob import glob

In [3]:
gpath = glob('animal_images/ゴリラ/*.jpg')
epath = glob('animal_images/ゾウ/*.jpg')
ppath = glob('animal_images/パンダ/*.jpg')
bpath = glob('animal_images/ホッキョクグマ/*.jpg')

In [4]:
n_total = len(gpath)
n_train = int(n_total * 0.7)
n_test = int(n_total * 0.3)

In [5]:
train_paths = gpath[:n_train] + epath[:n_train] + ppath[:n_train] + bpath[:n_train]
test_paths = gpath[n_train:] + epath[n_train:] + ppath[n_train:] + bpath[n_train:]

In [6]:
len(train_paths)

84

In [7]:
imsize = 256
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

In [8]:
# dataset = datasets.ImageFolder('animal_images', transform)
# dataset

In [9]:
import matplotlib.pyplot as plt

In [10]:
import torch
# import torchvision
# import torchvision.transforms as transforms

In [11]:
# import torch.nn as nn


import pytorch_lightning as pl
from pytorch_lightning import Trainer

In [14]:
from cnn_model import Net

In [15]:
# len(train), len(val)

In [16]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, path=None, transform=None):
        self.path = path
        self.transform = transform
        self.data = []
        self.labels = []

        #画像のラベルをファイル名から抜き出す
        for p in path:
            if 'ゴリラ' in p:
                label = 0
            elif 'ゾウ' in p:
                label = 1
            elif 'パンダ' in p:
                label = 2
            else:
                label = 3
            self.labels.append(label)

    def __len__(self):
        return len(list(self.path))

    def __getitem__(self, index):
        #index番目の画像を読み込み
        path = self.path[index]
        img = Image.open(path)

        #前処理
        img_transformed = self.transform(img)

        #index番目のラベルを読み込み
        label = self.labels[index]

        return img_transformed, label

In [17]:
# バッチサイズ
batch_size = 32
num_workers = 0



train = MyDataset(path=train_paths, transform=transform) 
val = MyDataset(path=test_paths, transform=transform)

# train[30]


train_loader = torch.utils.data.DataLoader(train, batch_size, num_workers=num_workers, shuffle=True)
val_loader = torch.utils.data.DataLoader(val, batch_size, num_workers=num_workers)

trainer = pl.Trainer(max_epochs=20)
model = Net()

trainer.fit(model, train_loader, val_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name | Type        | Params
-------------------------------------
0 | conv | Conv2d      | 84    
1 | pool | MaxPool2d   | 0     
2 | bn   | BatchNorm1d | 24 K  
3 | fc1  | Linear      | 1 M   
4 | fc2  | Linear      | 404   
Epoch 0:  60%|██████    | 3/5 [00:04<00:02,  1.43s/it, loss=6.463, v_num=14]
Validating: 0it [00:00, ?it/s]
Epoch 1:  60%|██████    | 3/5 [00:03<00:02,  1.04s/it, loss=4.458, v_num=14]
Validating: 0it [00:00, ?it/s]
Epoch 2:  60%|██████    | 3/5 [00:03<00:02,  1.05s/it, loss=3.493, v_num=14]
Validating: 0it [00:00, ?it/s]
Epoch 3:  60%|██████    | 3/5 [00:03<00:02,  1.01s/it, loss=3.047, v_num=14]
Validating: 0it [00:00, ?it/s]
Epoch 4:  60%|██████    | 3/5 [00:03<00:02,  1.08s/it, loss=2.582, v_num=14]
Validating: 0it [00:00, ?it/s]
Epoch 5:  60%|██████    | 3/5 [00:03<00:02,  1.04s/it, loss=2.265, v_num=14]
Validating: 0it [00:00, ?it/s]
Epoch 6:  60%|██████    | 3/5 [00:03<00:02,  1

1

In [19]:
trainer.callback_metrics

{'val_loss': tensor(32.0865),
 'val_acc': tensor(0.4219),
 'loss': tensor(0.1160)}

In [20]:
import torch.nn.functional as F

In [21]:
model.eval()
model.freeze()

In [23]:
torch.save(model.to('cpu').state_dict(), 'trained.pth')

In [28]:
import torch.onnx
from torch.autograd import Variable

In [41]:
y = Variable(train[0][0].unsqueeze(0))
torch.onnx.export(model, y, 'model.onnx', verbose=True)

graph(%input.1 : Float(1, 3, 128, 128),
      %conv.weight : Float(3, 3, 3, 3),
      %conv.bias : Float(3),
      %bn.weight : Float(12288),
      %bn.bias : Float(12288),
      %bn.running_mean : Float(12288),
      %bn.running_var : Float(12288),
      %fc1.weight : Float(100, 12288),
      %fc1.bias : Float(100),
      %fc2.weight : Float(4, 100),
      %fc2.bias : Float(4),
      %27 : Long(1)):
  %12 : Float(1, 3, 128, 128) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[3, 3], pads=[1, 1, 1, 1], strides=[1, 1]](%input.1, %conv.weight, %conv.bias) # C:\Users\image\anaconda3\lib\site-packages\torch\nn\modules\conv.py:350:0
  %13 : Float(1, 3, 128, 128) = onnx::Relu(%12) # C:\Users\image\anaconda3\lib\site-packages\torch\nn\functional.py:1063:0
  %14 : Float(1, 3, 64, 64) = onnx::MaxPool[kernel_shape=[2, 2], pads=[0, 0, 0, 0], strides=[2, 2]](%13) # C:\Users\image\anaconda3\lib\site-packages\torch\nn\functional.py:539:0
  %15 : Tensor = onnx::Shape(%14)
  %16 : Tensor = onnx: